<a href="https://colab.research.google.com/github/ParkEunHyeok/AI_Study/blob/main/NLP/seq2seq_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 7.7 MB/s 
     |████████████████████████████████| 448 kB 61.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# regex expression 적용하는 모듈
import re

# konlpy 형태소 분석기 사용하여 형태소 분석
from konlpy.tag import Okt

# Tokenize
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# One Hot Encoding
from tensorflow.keras.utils import to_categorical

# seq2seq Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import plot_model

# WARNING 무시
warnings.filterwarnings('ignore')

In [ ]:
# 구글 드라이브 연결
import os
from google.colab import drive
drive.mount('/content/gdrive/')
path = "gdrive/My Drive/Colab Notebooks/seq2seq"

Mounted at /content/gdrive/


In [ ]:
wellness = pd.read_csv("gdrive/My Drive/Colab Notebooks/HelloNewWorld/wellnessData.csv")
songys = pd.read_csv("gdrive/My Drive/Colab Notebooks/HelloNewWorld/songysData.csv")

In [ ]:
# 결측 row 제거하는 방법, 5232개
wellness.dropna(how='any')
wellness[:10]

,구분,유저,챗봇,라벨
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,0.0
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",0.0
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,"그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.",0.0
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0
5,감정/감정조절이상,작년 가을부터 감정조절이 잘 안 되는 거 같아.,"그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.",0.0
6,감정/감정조절이상,예전에 비해서 인내심이 너무 짧아진 거 같아.,요즈음 힘든 일이 많으신가봐요.,0.0
7,감정/감정조절이상,더 이상 혼자서는 감정조절을 못하겠어.,그럴 때는 어디에 털어놓아 보는건 어떨까요.,0.0
8,감정/감정조절이상,점점 나 자신을 컨트롤하지 못하는 기분이야.,그럴 때는 어디에 털어놓아 보는건 어떨까요.,0.0
9,감정/감정조절이상,나도 이러기 싫은데 내 마음대로 안돼.,저도 그 기분 이해해요. 많이 힘드시죠?,0.0


In [ ]:
texts = []
pairs = []

for sentence in wellness['유저']:
    texts.append(sentence)

for sentence in wellness['챗봇']:
    pairs.append(sentence)

list(zip(texts, pairs))[:5]

[('제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.',
  '감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.'),
 ('더 이상 내 감정을 내가 컨트롤 못 하겠어.', '저도 그 기분 이해해요. 많이 힘드시죠?'),
 ('하루종일 오르락내리락 롤러코스터 타는 기분이에요.', '그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.'),
 ('꼭 롤러코스터 타는 것 같아요.', '그럴 때 마음을 편하게 가지고, 기분 좋은 생각을 한번 해보는건 어떨까요.'),
 ('롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.', '저도 그 기분 이해해요. 많이 힘드시죠?')]

In [ ]:
# 램이 부족한 관계로 3000개만 학습함
texts = texts[:3000]
pairs = pairs[:3000]

In [ ]:
def clean_sentence(sentence):
    # 한글, 숫자를 제외한 모든 문자 제거
    sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    return sentence
  
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

'가나다12시 땡'

In [ ]:
okt = Okt()

# 형태소 변환해주는 함수
# morphs 함수 안에 변환한 한글 문장 입력
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

In [ ]:
def clean_and_morph(sentence, is_question=True):
    sentence = clean_sentence(sentence)
    sentence = process_morph(sentence)
    # Question 인 경우, Answer인 경우를 분기
    if is_question:
        return sentence
    else:
        # START 토큰 : decoder input, END 토큰 : decoder output
        return ('<START> ' + sentence, sentence + ' <END>')

In [ ]:
def preprocess(texts, pairs):
    questions = []
    answer_in = []
    answer_out = []

    # Question 전처리
    for text in texts:
        # 전처리와 morph 수행
        question = clean_and_morph(text, is_question=True)
        questions.append(question)

    # Answer 전처리
    for pair in pairs:
        # 전처리와 morph 수행
        in_, out_ = clean_and_morph(pair, is_question=False)
        answer_in.append(in_)
        answer_out.append(out_)
    
    return questions, answer_in, answer_out

In [ ]:
questions, answer_in, answer_out = preprocess(texts, pairs)

In [ ]:
questions[:5]

['제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요',
 '더 이상 내 감정 을 내 가 컨트롤 못 하겠어',
 '하루 종일 오르락내리락 롤러코스터 타는 기분 이에요',
 '꼭 롤러코스터 타는 것 같아요',
 '롤러코스터 타는 것 처럼 기분 이 왔다 갔다 해 요']

In [ ]:
answer_out[:5]

['감정 이 조절 이 안 될 때 만큼 힘들 때 는 없는 거 같아요 <END>',
 '저 도 그 기분 이해해 요 많이 힘드시죠 <END>',
 '그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 <END>',
 '그럴 때 마음 을 편하게 가지 고 기분 좋은 생각 을 한번 해보는건 어떨까 요 <END>',
 '저 도 그 기분 이해해 요 많이 힘드시죠 <END>']

In [ ]:
all_sentences = questions + answer_in + answer_out

In [ ]:
a = (' '.join(questions) + ' '.join(answer_in) + ' '.join(answer_out)).split()
len(set(a))

6866

In [ ]:
# Tokenizer 정의
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

# Word Index Vocabulary 만들기
tokenizer.fit_on_texts(all_sentences)

In [ ]:
for word, idx in tokenizer.word_index.items():
    print(f'{word}\t\t => \t{idx}')
    if idx > 10:
        break

<OOV>		 => 	1
이		 => 	2
<START>		 => 	3
<END>		 => 	4
을		 => 	5
가		 => 	6
도		 => 	7
것		 => 	8
요		 => 	9
은		 => 	10
에		 => 	11


In [ ]:
# 토큰 개수 확인
len(tokenizer.word_index)

6865

In [ ]:
# Text To Sequence Encoding
question_sequence = tokenizer.texts_to_sequences(questions)
answer_in_sequence = tokenizer.texts_to_sequences(answer_in)
answer_out_sequence = tokenizer.texts_to_sequences(answer_out)

In [ ]:
MAX_LENGTH = 20

question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_in_padded = pad_sequences(answer_in_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
answer_out_padded = pad_sequences(answer_out_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')

In [ ]:
print(question_padded.shape)
print(answer_in_padded.shape, answer_out_padded.shape)

(3000, 20)
(3000, 20) (3000, 20)


In [ ]:
# Encoder
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps, name='Embedding')
        self.dropout = Dropout(0.3, name='Dropout')
        # (attention) return_sequences=True 추가
        self.lstm = LSTM(units, return_state=True, return_sequences=True, name='LSTM')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # (attention) x return 추가
        return x, [hidden_state, cell_state]

In [ ]:
# Decoder
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps, name='Embedding')
        self.dropout = Dropout(0.3, name='Dropout')
        self.lstm = LSTM(units, 
                         return_state=True, 
                         return_sequences=True, 
                         name='LSTM'
                        )
        self.attention = Attention(name='Attention')
        self.dense = Dense(VOCAB_SIZE, activation='softmax', name='Dense')
    
    def call(self, inputs, initial_state):
        # (attention) encoder_inputs 추가
        encoder_inputs, decoder_inputs = inputs
        x = self.embedding(decoder_inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        
        # (attention) key_value, attention_matrix 추가
        # 이전 hidden_state의 값을 concat으로 만들어 vector를 생성합니다.        
        key_value = tf.concat([initial_state[0][:, tf.newaxis, :], x[:, :-1, :]], axis=1)        
        # 이전 hidden_state의 값을 concat으로 만든 vector와 encoder에서 나온 출력 값들로 attention을 구합니다.
        attention_matrix = self.attention([key_value, encoder_inputs])
        # 위에서 구한 attention_matrix와 decoder의 출력 값을 concat 합니다.
        x = tf.concat([x, attention_matrix], axis=-1)
        
        x = self.dense(x)
        return x, hidden_state, cell_state

In [ ]:
# seq2seq 모델
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps
        
        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)
        
        
    def call(self, inputs, training=True):
        if training:
            encoder_inputs, decoder_inputs = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(encoder_inputs)
            # (attention) decoder 입력 값 수정
            decoder_outputs, _, _ = self.decoder((encoder_outputs, decoder_inputs), initial_state=context_vector)
            return decoder_outputs
        else:
            x = inputs
            # (attention) encoder 출력 값 수정
            encoder_outputs, context_vector = self.encoder(x)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)
            
            for i in tf.range(self.time_steps):
                decoder_output, decoder_hidden, decoder_cell = self.decoder((encoder_outputs, target_seq), initial_state=context_vector)
                decoder_output = tf.cast(tf.argmax(decoder_output, axis=-1), dtype=tf.int32)
                decoder_output = tf.reshape(decoder_output, shape=(1, 1))
                results = results.write(i, decoder_output)
                
                if decoder_output == self.end_token:
                    break
                    
                target_seq = decoder_output
                context_vector = [decoder_hidden, decoder_cell]
                
            return tf.reshape(results.stack(), shape=(1, self.time_steps))

In [ ]:
# 단어 별 One-Hot Encoding
# answer를 인코딩함

VOCAB_SIZE = len(tokenizer.word_index)+1

def convert_to_one_hot(padded):
    # 원핫인코딩 초기화
    one_hot_vector = np.zeros((len(answer_out_padded), MAX_LENGTH, VOCAB_SIZE))

    # 디코더 목표를 원핫인코딩으로 변환
    # 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
    for i, sequence in enumerate(answer_out_padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1

    return one_hot_vector

In [ ]:
answer_in_one_hot = convert_to_one_hot(answer_in_padded)
answer_out_one_hot = convert_to_one_hot(answer_out_padded)

In [ ]:
answer_in_one_hot[0].shape, answer_in_one_hot[0].shape

((20, 6866), (20, 6866))

In [ ]:
# sequence to index
def convert_index_to_text(indexs, end_token): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == end_token:
            # 끝 단어이므로 예측 중비
            break;
        # 사전에 존재하는 단어의 경우 단어 추가
        if index > 0 and tokenizer.index_word[index] is not None:
            sentence += tokenizer.index_word[index]
        else:
        # 사전에 없는 인덱스면 빈 문자열 추가
            sentence += ''
            
        # 빈칸 추가
        sentence += ' '
    return sentence

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH
START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']

UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index)+1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3

In [ ]:
# 모델 분산 학습 할 때 체크포인트 저장
checkpoint_path = 'gdrive/My Drive/Colab Notebooks/seq2seq/training_checkpoint-6.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True,
                             save_best_only=True, 
                             monitor='loss', 
                             verbose=1
                            )

In [ ]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
strategy.num_replicas_in_sync

1

In [ ]:
# 분산 환경 적용시
with strategy.scope():
    seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
    seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [ ]:
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
# 연속하여 학습시 체크포인트를 로드하여 이어서 학습합니다.
#seq2seq.load_weights(checkpoint_path)

In [ ]:
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    # 변환된 인덱스를 문장으로 변환
    results = np.asarray(results).reshape(-1)
    return results

In [ ]:
# seq2seq 모델 학습
for epoch in range(10):
    seq2seq.fit([question_padded, answer_in_padded],
                answer_out_one_hot,
                epochs=10,
                batch_size=16, 
                callbacks=[checkpoint]
               )
    # 랜덤한 샘플 번호 추출
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    # 예측 성능 테스트
    for idx in samples:
        question_inputs = question_padded[idx]
        # 문장 예측
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        
        # 변환된 인덱스를 문장으로 변환
        results = convert_index_to_text(results, END_TOKEN)
        
        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

Epoch 1/10


In [ ]:
# 랜덤한 샘플 번호 추출
samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

# 예측 성능 테스트
for idx in samples:
    question_inputs = question_padded[idx]
    # 문장 예측
    results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
    
    # 변환된 인덱스를 문장으로 변환
    results = convert_index_to_text(results, END_TOKEN)
    
    print(f'Q: {questions[idx]}')
    print(f'A: {results}\n')
    print()

In [ ]:
seq2seq.summary()

In [ ]:
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
#tf.keras.models.save_model(seq2seq,'gdrive/My Drive/Colab Notebooks/seq2seq/seq2seq_model.h5')
seq2seq.save('gdrive/My Drive/Colab Notebooks/seq2seq/seq2seq_model.h5')

In [ ]:
# 자연어 (질문 입력) 대한 전처리 함수
def make_question(sentence):
    sentence = clean_and_morph(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

In [ ]:
make_question('오늘 날씨가 정말 화창합니다')

In [ ]:
# 챗봇 테스트, 입력받으면 전처리함
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = convert_index_to_text(results, END_TOKEN)
    return results

In [ ]:
# 사용자 테스트
while True:
    user_input = input('<< 말을 걸어 보세요!\n')
    if user_input == 'q':
        break
    print('>> 챗봇 응답: {}'.format(run_chatbot(user_input)))